# Python Initialization Configuration

3.8

Python can be initialized with :c`Py_InitializeFromConfig` and the
:c`PyConfig` structure. It can be preinitialized with
:c`Py_PreInitialize` and the :c`PyPreConfig` structure.

There are two kinds of configuration:

-   The `Python Configuration <init-python-config>` can be used to build
    a customized Python which behaves as the regular Python. For
    example, environment variables and command line arguments are used
    to configure Python.
-   The `Isolated Configuration <init-isolated-conf>` can be used to
    embed Python into an application. It isolates Python from the
    system. For example, environment variables are ignored, the LC_CTYPE
    locale is left unchanged and no signal handler is registered.

The :c`Py_RunMain` function can be used to write a customized Python
program.

See also `Initialization, Finalization, and Threads <initialization>`.

`587` "Python Initialization Configuration".

## Example

Example of customized Python always running in isolated mode:

``` c
int main(int argc, char **argv)
{
    PyStatus status;

    PyConfig config;
    PyConfig_InitPythonConfig(&config);
    config.isolated = 1;

    /* Decode command line arguments.
       Implicitly preinitialize Python (in isolated mode). */
    status = PyConfig_SetBytesArgv(&config, argc, argv);
    if (PyStatus_Exception(status)) {
        goto exception;
    }

    status = Py_InitializeFromConfig(&config);
    if (PyStatus_Exception(status)) {
        goto exception;
    }
    PyConfig_Clear(&config);

    return Py_RunMain();

exception:
    PyConfig_Clear(&config);
    if (PyStatus_IsExit(status)) {
        return status.exitcode;
    }
    /* Display the error message and exit the process with
       non-zero exit code */
    Py_ExitStatusException(status);
}
```

## PyWideStringList

> List of `wchar_t*` strings.
>
> If *length* is non-zero, *items* must be non-`NULL` and all strings
> must be non-`NULL`.
>
> Methods:
>
> > Append *item* to *list*.
> >
> > Python must be preinitialized to call this function.
>
> > Insert *item* into *list* at *index*.
> >
> > If *index* is greater than or equal to *list* length, append *item*
> > to *list*.
> >
> > *index* must be greater than or equal to `0`.
> >
> > Python must be preinitialized to call this function.
>
> Structure fields:
>
> > List length.
>
> > List items.

## PyStatus

> Structure to store an initialization function status: success, error
> or exit.
>
> For an error, it can store the C function name which created the
> error.
>
> Structure fields:
>
> > Exit code. Argument passed to `exit()`.
>
> > Error message.
>
> > Name of the function which created an error, can be `NULL`.
>
> Functions to create a status:
>
> > Success.
>
> > Initialization error with a message.
> >
> > *err_msg* must not be `NULL`.
>
> > Memory allocation failure (out of memory).
>
> > Exit Python with the specified exit code.
>
> Functions to handle a status:
>
> > Is the status an error or an exit? If true, the exception must be
> > handled; by calling :c`Py_ExitStatusException` for example.
>
> > Is the result an error?
>
> > Is the result an exit?
>
> > Call `exit(exitcode)` if *status* is an exit. Print the error
> > message and exit with a non-zero exit code if *status* is an error.
> > Must only be called if `PyStatus_Exception(status)` is non-zero.

Note

Internally, Python uses macros which set `PyStatus.func`, whereas
functions to create a status set `func` to `NULL`.

Example:

``` c
PyStatus alloc(void **ptr, size_t size)
{
    *ptr = PyMem_RawMalloc(size);
    if (*ptr == NULL) {
        return PyStatus_NoMemory();
    }
    return PyStatus_Ok();
}

int main(int argc, char **argv)
{
    void *ptr;
    PyStatus status = alloc(&ptr, 16);
    if (PyStatus_Exception(status)) {
        Py_ExitStatusException(status);
    }
    PyMem_Free(ptr);
    return 0;
}
```

## PyPreConfig

> Structure used to preinitialize Python.
>
> Function to initialize a preconfiguration:
>
> > Initialize the preconfiguration with `Python Configuration
> > <init-python-config>`.
>
> > Initialize the preconfiguration with `Isolated Configuration
> > <init-isolated-conf>`.
>
> Structure fields:
>
> > Name of the Python memory allocators:
> >
> > -   `PYMEM_ALLOCATOR_NOT_SET` (`0`): don't change memory allocators
> >     (use defaults).
> > -   `PYMEM_ALLOCATOR_DEFAULT` (`1`): `default memory allocators
> >     <default-memory-allocators>`.
> > -   `PYMEM_ALLOCATOR_DEBUG` (`2`): `default memory allocators
> >     <default-memory-allocators>` with `debug hooks
> >     <pymem-debug-hooks>`.
> > -   `PYMEM_ALLOCATOR_MALLOC` (`3`): use `malloc()` of the C library.
> > -   `PYMEM_ALLOCATOR_MALLOC_DEBUG` (`4`): force usage of `malloc()`
> >     with `debug hooks <pymem-debug-hooks>`.
> > -   `PYMEM_ALLOCATOR_PYMALLOC` (`5`): `Python pymalloc memory
> >     allocator <pymalloc>`.
> > -   `PYMEM_ALLOCATOR_PYMALLOC_DEBUG` (`6`): `Python pymalloc
> >     memory allocator <pymalloc>` with `debug hooks
> >     <pymem-debug-hooks>`.
> >
> > `PYMEM_ALLOCATOR_PYMALLOC` and `PYMEM_ALLOCATOR_PYMALLOC_DEBUG` are
> > not supported if Python is `configured using --without-pymalloc
> > <--without-pymalloc>`.
> >
> > See `Memory Management <memory>`.
> >
> > Default: `PYMEM_ALLOCATOR_NOT_SET`.
>
> > Set the LC_CTYPE locale to the user preferred locale?
> >
> > If equals to `0`, set :c`~PyPreConfig.coerce_c_locale` and
> > :c`~PyPreConfig.coerce_c_locale_warn` members to `0`.
> >
> > See the `locale encoding`.
> >
> > Default: `1` in Python config, `0` in isolated config.
>
> > If equals to `2`, coerce the C locale.
> >
> > If equals to `1`, read the LC_CTYPE locale to decide if it should be
> > coerced.
> >
> > See the `locale encoding`.
> >
> > Default: `-1` in Python config, `0` in isolated config.
>
> > If non-zero, emit a warning if the C locale is coerced.
> >
> > Default: `-1` in Python config, `0` in isolated config.
>
> > `Python Development Mode <devmode>`: see :c`PyConfig.dev_mode`.
> >
> > Default: `-1` in Python mode, `0` in isolated mode.
>
> > Isolated mode: see :c`PyConfig.isolated`.
> >
> > Default: `0` in Python mode, `1` in isolated mode.
>
> > If non-zero:
> >
> > -   Set :c`PyPreConfig.utf8_mode` to `0`,
> > -   Set :c`PyConfig.filesystem_encoding` to `"mbcs"`,
> > -   Set :c`PyConfig.filesystem_errors` to `"replace"`.
> >
> > Initialized the from `PYTHONLEGACYWINDOWSFSENCODING` environment
> > variable value.
> >
> > Only available on Windows. `#ifdef MS_WINDOWS` macro can be used for
> > Windows specific code.
> >
> > Default: `0`.
>
> > If non-zero, :c`Py_PreInitializeFromArgs` and
> > :c`Py_PreInitializeFromBytesArgs` parse their `argv` argument the
> > same way the regular Python parses command line arguments: see
> > `Command Line Arguments <using-on-cmdline>`.
> >
> > Default: `1` in Python config, `0` in isolated config.
>
> > Use `environment variables <using-on-envvars>`? See
> > :c`PyConfig.use_environment`.
> >
> > Default: `1` in Python config and `0` in isolated config.
>
> > If non-zero, enable the `Python UTF-8 Mode <utf8-mode>`.
> >
> > Set to `0` or `1` by the `-X utf8 <-X>` command line option and the
> > `PYTHONUTF8` environment variable.
> >
> > Also set to `1` if the `LC_CTYPE` locale is `C` or `POSIX`.
> >
> > Default: `-1` in Python config and `0` in isolated config.

## Preinitialize Python with PyPreConfig

The preinitialization of Python:

-   Set the Python memory allocators (:c`PyPreConfig.allocator`)
-   Configure the LC_CTYPE locale (`locale encoding`)
-   Set the `Python UTF-8 Mode <utf8-mode>` (:c`PyPreConfig.utf8_mode`)

The current preconfiguration (`PyPreConfig` type) is stored in
`_PyRuntime.preconfig`.

Functions to preinitialize Python:

> Preinitialize Python from *preconfig* preconfiguration.
>
> *preconfig* must not be `NULL`.

> Preinitialize Python from *preconfig* preconfiguration.
>
> Parse *argv* command line arguments (bytes strings) if
> :c`~PyPreConfig.parse_argv` of *preconfig* is non-zero.
>
> *preconfig* must not be `NULL`.

> Preinitialize Python from *preconfig* preconfiguration.
>
> Parse *argv* command line arguments (wide strings) if
> :c`~PyPreConfig.parse_argv` of *preconfig* is non-zero.
>
> *preconfig* must not be `NULL`.

The caller is responsible to handle exceptions (error or exit) using
:c`PyStatus_Exception` and :c`Py_ExitStatusException`.

For `Python Configuration <init-python-config>`
(:c`PyPreConfig_InitPythonConfig`), if Python is initialized with
command line arguments, the command line arguments must also be passed
to preinitialize Python, since they have an effect on the
pre-configuration like encodings. For example, the `-X utf8 <-X>`
command line option enables the `Python UTF-8 Mode <utf8-mode>`.

`PyMem_SetAllocator()` can be called after :c`Py_PreInitialize` and
before :c`Py_InitializeFromConfig` to install a custom memory allocator.
It can be called before :c`Py_PreInitialize` if
:c`PyPreConfig.allocator` is set to `PYMEM_ALLOCATOR_NOT_SET`.

Python memory allocation functions like :c`PyMem_RawMalloc` must not be
used before the Python preinitialization, whereas calling directly
`malloc()` and `free()` is always safe. :c`Py_DecodeLocale` must not be
called before the Python preinitialization.

Example using the preinitialization to enable the
`Python UTF-8 Mode <utf8-mode>`:

``` c
PyStatus status;
PyPreConfig preconfig;
PyPreConfig_InitPythonConfig(&preconfig);

preconfig.utf8_mode = 1;

status = Py_PreInitialize(&preconfig);
if (PyStatus_Exception(status)) {
    Py_ExitStatusException(status);
}

/* at this point, Python speaks UTF-8 */

Py_Initialize();
/* ... use Python API here ... */
Py_Finalize();
```

## PyConfig

> Structure containing most parameters to configure Python.
>
> When done, the :c`PyConfig_Clear` function must be used to release the
> configuration memory.
>
> Structure methods:
>
> > Initialize configuration with the `Python Configuration
> > <init-python-config>`.
>
> > Initialize configuration with the `Isolated Configuration
> > <init-isolated-conf>`.
>
> > Copy the wide character string *str* into `*config_str`.
> >
> > `Preinitialize Python <c-preinit>` if needed.
>
> > Decode *str* using :c`Py_DecodeLocale` and set the result into
> > `*config_str`.
> >
> > `Preinitialize Python <c-preinit>` if needed.
>
> > Set command line arguments (:c`~PyConfig.argv` member of *config*)
> > from the *argv* list of wide character strings.
> >
> > `Preinitialize Python <c-preinit>` if needed.
>
> > Set command line arguments (:c`~PyConfig.argv` member of *config*)
> > from the *argv* list of bytes strings. Decode bytes using
> > :c`Py_DecodeLocale`.
> >
> > `Preinitialize Python <c-preinit>` if needed.
>
> > Set the list of wide strings *list* to *length* and *items*.
> >
> > `Preinitialize Python <c-preinit>` if needed.
>
> > Read all Python configuration.
> >
> > Fields which are already initialized are left unchanged.
> >
> > Fields for `path configuration <init-path-config>` are no longer
> > calculated or modified when calling this function, as of Python
> > 3.11.
> >
> > The :c`PyConfig_Read` function only parses :c`PyConfig.argv`
> > arguments once: :c`PyConfig.parse_argv` is set to `2` after
> > arguments are parsed. Since Python arguments are strippped from
> > :c`PyConfig.argv`, parsing arguments twice would parse the
> > application options as Python options.
> >
> > `Preinitialize Python <c-preinit>` if needed.
> >
> > 3.10 The :c`PyConfig.argv` arguments are now only parsed once,
> > :c`PyConfig.parse_argv` is set to `2` after arguments are parsed,
> > and arguments are only parsed if :c`PyConfig.parse_argv` equals `1`.
> >
> > 3.11 :c`PyConfig_Read` no longer calculates all paths, and so fields
> > listed under `Python Path Configuration <init-path-config>` may no
> > longer be updated until :c`Py_InitializeFromConfig` is called.
>
> > Release configuration memory.
>
> Most `PyConfig` methods `preinitialize Python <c-preinit>` if needed.
> In that case, the Python preinitialization configuration
> (:c`PyPreConfig`) in based on the :c`PyConfig`. If configuration
> fields which are in common with :c`PyPreConfig` are tuned, they must
> be set before calling a :c`PyConfig` method:
>
> -   :c`PyConfig.dev_mode`
> -   :c`PyConfig.isolated`
> -   :c`PyConfig.parse_argv`
> -   :c`PyConfig.use_environment`
>
> Moreover, if :c`PyConfig_SetArgv` or :c`PyConfig_SetBytesArgv` is
> used, this method must be called before other methods, since the
> preinitialization configuration depends on command line arguments (if
> :c`parse_argv` is non-zero).
>
> The caller of these methods is responsible to handle exceptions (error
> or exit) using `PyStatus_Exception()` and `Py_ExitStatusException()`.
>
> Structure fields:
>
> > Command line arguments: `sys.argv`.
> >
> > Set :c`~PyConfig.parse_argv` to `1` to parse :c`~PyConfig.argv` the
> > same way the regular Python parses Python command line arguments and
> > then to strip Python arguments from :c`~PyConfig.argv`.
> >
> > If :c`~PyConfig.argv` is empty, an empty string is added to ensure
> > that `sys.argv` always exists and is never empty.
> >
> > Default: `NULL`.
> >
> > See also the :c`~PyConfig.orig_argv` member.
>
> > If equals to zero, `Py_RunMain()` prepends a potentially unsafe path
> > to `sys.path` at startup:
> >
> > -   If :c`argv[0] <PyConfig.argv>` is equal to `L"-m"`
> >     (`python -m module`), prepend the current working directory.
> > -   If running a script (`python script.py`), prepend the script's
> >     directory. If it's a symbolic link, resolve symbolic links.
> > -   Otherwise (`python -c code` and `python`), prepend an empty
> >     string, which means the current working directory.
> >
> > Set to `1` by the `-P` command line option and the `PYTHONSAFEPATH`
> > environment variable.
> >
> > Default: `0` in Python config, `1` in isolated config.
> >
> > 3.11
>
> > `sys.base_exec_prefix`.
> >
> > Default: `NULL`.
> >
> > Part of the `Python Path Configuration <init-path-config>` output.
>
> > Python base executable: `sys._base_executable`.
> >
> > Set by the `__PYVENV_LAUNCHER__` environment variable.
> >
> > Set from :c`PyConfig.executable` if `NULL`.
> >
> > Default: `NULL`.
> >
> > Part of the `Python Path Configuration <init-path-config>` output.
>
> > `sys.base_prefix`.
> >
> > Default: `NULL`.
> >
> > Part of the `Python Path Configuration <init-path-config>` output.
>
> > If equals to `0` and :c`~PyConfig.configure_c_stdio` is non-zero,
> > disable buffering on the C streams stdout and stderr.
> >
> > Set to `0` by the `-u` command line option and the
> > `PYTHONUNBUFFERED` environment variable.
> >
> > stdin is always opened in buffered mode.
> >
> > Default: `1`.
>
> > If equals to `1`, issue a warning when comparing `bytes` or
> > `bytearray` with `str`, or comparing `bytes` with `int`.
> >
> > If equal or greater to `2`, raise a `BytesWarning` exception in
> > these cases.
> >
> > Incremented by the `-b` command line option.
> >
> > Default: `0`.
>
> > If non-zero, emit a `EncodingWarning` warning when
> > `io.TextIOWrapper` uses its default encoding. See
> > `io-encoding-warning` for details.
> >
> > Default: `0`.
> >
> > 3.10
>
> > If equals to `0`, disables the inclusion of the end line and column
> > mappings in code objects. Also disables traceback printing carets to
> > specific error locations.
> >
> > Set to `0` by the `PYTHONNODEBUGRANGES` environment variable and by
> > the `-X no_debug_ranges <-X>` command line option.
> >
> > Default: `1`.
> >
> > 3.11
>
> > Control the validation behavior of hash-based `.pyc` files: value of
> > the `--check-hash-based-pycs` command line option.
> >
> > Valid values:
> >
> > -   `L"always"`: Hash the source file for invalidation regardless of
> >     value of the 'check_source' flag.
> > -   `L"never"`: Assume that hash-based pycs always are valid.
> > -   `L"default"`: The 'check_source' flag in hash-based pycs
> >     determines invalidation.
> >
> > Default: `L"default"`.
> >
> > See also `552` "Deterministic pycs".
>
> > If non-zero, configure C standard streams:
> >
> > -   On Windows, set the binary mode (`O_BINARY`) on stdin, stdout
> >     and stderr.
> > -   If :c`~PyConfig.buffered_stdio` equals zero, disable buffering
> >     of stdin, stdout and stderr streams.
> > -   If :c`~PyConfig.interactive` is non-zero, enable stream
> >     buffering on stdin and stdout (only stdout on Windows).
> >
> > Default: `1` in Python config, `0` in isolated config.
>
> > If non-zero, enable the `Python Development Mode <devmode>`.
> >
> > Set to `1` by the `-X dev <-X>` option and the `PYTHONDEVMODE`
> > environment variable.
> >
> > Default: `-1` in Python mode, `0` in isolated mode.
>
> > Dump Python references?
> >
> > If non-zero, dump all objects which are still alive at exit.
> >
> > Set to `1` by the `PYTHONDUMPREFS` environment variable.
> >
> > Need a special build of Python with the `Py_TRACE_REFS` macro
> > defined: see the
> > `configure --with-trace-refs option <--with-trace-refs>`.
> >
> > Default: `0`.
>
> > The site-specific directory prefix where the platform-dependent
> > Python files are installed: `sys.exec_prefix`.
> >
> > Default: `NULL`.
> >
> > Part of the `Python Path Configuration <init-path-config>` output.
>
> > The absolute path of the executable binary for the Python
> > interpreter: `sys.executable`.
> >
> > Default: `NULL`.
> >
> > Part of the `Python Path Configuration <init-path-config>` output.
>
> > Enable faulthandler?
> >
> > If non-zero, call `faulthandler.enable` at startup.
> >
> > Set to `1` by `-X faulthandler <-X>` and the `PYTHONFAULTHANDLER`
> > environment variable.
> >
> > Default: `-1` in Python mode, `0` in isolated mode.
>
> > `Filesystem encoding <filesystem encoding and error handler>`:
> > `sys.getfilesystemencoding`.
> >
> > On macOS, Android and VxWorks: use `"utf-8"` by default.
> >
> > On Windows: use `"utf-8"` by default, or `"mbcs"` if
> > :c`~PyPreConfig.legacy_windows_fs_encoding` of :c`PyPreConfig` is
> > non-zero.
> >
> > Default encoding on other platforms:
> >
> > -   `"utf-8"` if :c`PyPreConfig.utf8_mode` is non-zero.
> > -   `"ascii"` if Python detects that `nl_langinfo(CODESET)`
> >     announces the ASCII encoding (or Roman8 encoding on HP-UX),
> >     whereas the `mbstowcs()` function decodes from a different
> >     encoding (usually Latin1).
> > -   `"utf-8"` if `nl_langinfo(CODESET)` returns an empty string.
> > -   Otherwise, use the `locale encoding`: `nl_langinfo(CODESET)`
> >     result.
> >
> > At Python startup, the encoding name is normalized to the Python
> > codec name. For example, `"ANSI_X3.4-1968"` is replaced with
> > `"ascii"`.
> >
> > See also the :c`~PyConfig.filesystem_errors` member.
>
> > `Filesystem error handler <filesystem encoding and error handler>`:
> > `sys.getfilesystemencodeerrors`.
> >
> > On Windows: use `"surrogatepass"` by default, or `"replace"` if
> > :c`~PyPreConfig.legacy_windows_fs_encoding` of :c`PyPreConfig` is
> > non-zero.
> >
> > On other platforms: use `"surrogateescape"` by default.
> >
> > Supported error handlers:
> >
> > -   `"strict"`
> > -   `"surrogateescape"`
> > -   `"surrogatepass"` (only supported with the UTF-8 encoding)
> >
> > See also the :c`~PyConfig.filesystem_encoding` member.
>
> > Randomized hash function seed.
> >
> > If :c`~PyConfig.use_hash_seed` is zero, a seed is chosen randomly at
> > Python startup, and :c`~PyConfig.hash_seed` is ignored.
> >
> > Set by the `PYTHONHASHSEED` environment variable.
> >
> > Default *use_hash_seed* value: `-1` in Python mode, `0` in isolated
> > mode.
>
> > Python home directory.
> >
> > If :c`Py_SetPythonHome` has been called, use its argument if it is
> > not `NULL`.
> >
> > Set by the `PYTHONHOME` environment variable.
> >
> > Default: `NULL`.
> >
> > Part of the `Python Path Configuration <init-path-config>` input.
>
> > If non-zero, profile import time.
> >
> > Set the `1` by the `-X importtime <-X>` option and the
> > `PYTHONPROFILEIMPORTTIME` environment variable.
> >
> > Default: `0`.
>
> > Enter interactive mode after executing a script or a command.
> >
> > If greater than `0`, enable inspect: when a script is passed as
> > first argument or the -c option is used, enter interactive mode
> > after executing the script or the command, even when `sys.stdin`
> > does not appear to be a terminal.
> >
> > Incremented by the `-i` command line option. Set to `1` if the
> > `PYTHONINSPECT` environment variable is non-empty.
> >
> > Default: `0`.
>
> > Install Python signal handlers?
> >
> > Default: `1` in Python mode, `0` in isolated mode.
>
> > If greater than `0`, enable the interactive mode (REPL).
> >
> > Incremented by the `-i` command line option.
> >
> > Default: `0`.
>
> > If greater than `0`, enable isolated mode:
> >
> > -   Set :c`~PyConfig.safe_path` to `1`: don't prepend a potentially
> >     unsafe path to `sys.path` at Python startup, such as the current
> >     directory, the script's directory or an empty string.
> > -   Set :c`~PyConfig.use_environment` to `0`: ignore `PYTHON`
> >     environment variables.
> > -   Set :c`~PyConfig.user_site_directory` to `0`: don't add the user
> >     site directory to `sys.path`.
> > -   Python REPL doesn't import `readline` nor enable default
> >     readline configuration on interactive prompts.
> >
> > Set to `1` by the `-I` command line option.
> >
> > Default: `0` in Python mode, `1` in isolated mode.
> >
> > See also the `Isolated Configuration <init-isolated-conf>` and
> > :c`PyPreConfig.isolated`.
>
> > If non-zero, use `io.FileIO` instead of `io.WindowsConsoleIO` for
> > `sys.stdin`, `sys.stdout` and `sys.stderr`.
> >
> > Set to `1` if the `PYTHONLEGACYWINDOWSSTDIO` environment variable is
> > set to a non-empty string.
> >
> > Only available on Windows. `#ifdef MS_WINDOWS` macro can be used for
> > Windows specific code.
> >
> > Default: `0`.
> >
> > See also the `528` (Change Windows console encoding to UTF-8).
>
> > If non-zero, dump statistics on `Python pymalloc memory allocator
> > <pymalloc>` at exit.
> >
> > Set to `1` by the `PYTHONMALLOCSTATS` environment variable.
> >
> > The option is ignored if Python is `configured using
> > the --without-pymalloc option <--without-pymalloc>`.
> >
> > Default: `0`.
>
> > Platform library directory name: `sys.platlibdir`.
> >
> > Set by the `PYTHONPLATLIBDIR` environment variable.
> >
> > Default: value of the `PLATLIBDIR` macro which is set by the
> > `configure --with-platlibdir option <--with-platlibdir>` (default:
> > `"lib"`, or `"DLLs"` on Windows).
> >
> > Part of the `Python Path Configuration <init-path-config>` input.
> >
> > 3.9
> >
> > 3.11 This macro is now used on Windows to locate the standard
> > library extension modules, typically under `DLLs`. However, for
> > compatibility, note that this value is ignored for any non-standard
> > layouts, including in-tree builds and virtual environments.
>
> > Module search paths (`sys.path`) as a string separated by `DELIM`
> > (`os.path.pathsep`).
> >
> > Set by the `PYTHONPATH` environment variable.
> >
> > Default: `NULL`.
> >
> > Part of the `Python Path Configuration <init-path-config>` input.
>
> > Module search paths: `sys.path`.
> >
> > If :c`~PyConfig.module_search_paths_set` is equal to `0`,
> > :c`Py_InitializeFromConfig` will replace
> > :c`~PyConfig.module_search_paths` and sets
> > :c`~PyConfig.module_search_paths_set` to `1`.
> >
> > Default: empty list (`module_search_paths`) and `0`
> > (`module_search_paths_set`).
> >
> > Part of the `Python Path Configuration <init-path-config>` output.
>
> > Compilation optimization level:
> >
> > -   `0`: Peephole optimizer, set `__debug__` to `True`.
> > -   `1`: Level 0, remove assertions, set `__debug__` to `False`.
> > -   `2`: Level 1, strip docstrings.
> >
> > Incremented by the `-O` command line option. Set to the
> > `PYTHONOPTIMIZE` environment variable value.
> >
> > Default: `0`.
>
> > The list of the original command line arguments passed to the Python
> > executable: `sys.orig_argv`.
> >
> > If :c`~PyConfig.orig_argv` list is empty and :c`~PyConfig.argv` is
> > not a list only containing an empty string, :c`PyConfig_Read` copies
> > :c`~PyConfig.argv` into :c`~PyConfig.orig_argv` before modifying
> > :c`~PyConfig.argv` (if :c`~PyConfig.parse_argv` is non-zero).
> >
> > See also the :c`~PyConfig.argv` member and the :c`Py_GetArgcArgv`
> > function.
> >
> > Default: empty list.
> >
> > 3.10
>
> > Parse command line arguments?
> >
> > If equals to `1`, parse :c`~PyConfig.argv` the same way the regular
> > Python parses `command line arguments <using-on-cmdline>`, and strip
> > Python arguments from :c`~PyConfig.argv`.
> >
> > The :c`PyConfig_Read` function only parses :c`PyConfig.argv`
> > arguments once: :c`PyConfig.parse_argv` is set to `2` after
> > arguments are parsed. Since Python arguments are strippped from
> > :c`PyConfig.argv`, parsing arguments twice would parse the
> > application options as Python options.
> >
> > Default: `1` in Python mode, `0` in isolated mode.
> >
> > 3.10 The :c`PyConfig.argv` arguments are now only parsed if
> > :c`PyConfig.parse_argv` equals to `1`.
>
> > Parser debug mode. If greater than `0`, turn on parser debugging
> > output (for expert only, depending on compilation options).
> >
> > Incremented by the `-d` command line option. Set to the
> > `PYTHONDEBUG` environment variable value.
> >
> > Need a `debug build of Python <debug-build>` (the `Py_DEBUG` macro
> > must be defined).
> >
> > Default: `0`.
>
> > If non-zero, calculation of path configuration is allowed to log
> > warnings into `stderr`. If equals to `0`, suppress these warnings.
> >
> > Default: `1` in Python mode, `0` in isolated mode.
> >
> > Part of the `Python Path Configuration <init-path-config>` input.
> >
> > 3.11 Now also applies on Windows.
>
> > The site-specific directory prefix where the platform independent
> > Python files are installed: `sys.prefix`.
> >
> > Default: `NULL`.
> >
> > Part of the `Python Path Configuration <init-path-config>` output.
>
> > Program name used to initialize :c`~PyConfig.executable` and in
> > early error messages during Python initialization.
> >
> > -   If `Py_SetProgramName` has been called, use its argument.
> > -   On macOS, use `PYTHONEXECUTABLE` environment variable if set.
> > -   If the `WITH_NEXT_FRAMEWORK` macro is defined, use
> >     `__PYVENV_LAUNCHER__` environment variable if set.
> > -   Use `argv[0]` of :c`~PyConfig.argv` if available and non-empty.
> > -   Otherwise, use `L"python"` on Windows, or `L"python3"` on other
> >     platforms.
> >
> > Default: `NULL`.
> >
> > Part of the `Python Path Configuration <init-path-config>` input.
>
> > Directory where cached `.pyc` files are written:
> > `sys.pycache_prefix`.
> >
> > Set by the `-X pycache_prefix=PATH <-X>` command line option and the
> > `PYTHONPYCACHEPREFIX` environment variable.
> >
> > If `NULL`, `sys.pycache_prefix` is set to `None`.
> >
> > Default: `NULL`.
>
> > Quiet mode. If greater than `0`, don't display the copyright and
> > version at Python startup in interactive mode.
> >
> > Incremented by the `-q` command line option.
> >
> > Default: `0`.
>
> > Value of the `-c` command line option.
> >
> > Used by :c`Py_RunMain`.
> >
> > Default: `NULL`.
>
> > Filename passed on the command line: trailing command line argument
> > without `-c` or `-m`. It is used by the :c`Py_RunMain` function.
> >
> > For example, it is set to `script.py` by the `python3 script.py arg`
> > command line.
> >
> > See also the :c`PyConfig.skip_source_first_line` option.
> >
> > Default: `NULL`.
>
> > Value of the `-m` command line option.
> >
> > Used by :c`Py_RunMain`.
> >
> > Default: `NULL`.
>
> > Show total reference count at exit?
> >
> > Set to `1` by `-X showrefcount <-X>` command line option.
> >
> > Need a `debug build of Python <debug-build>` (the `Py_REF_DEBUG`
> > macro must be defined).
> >
> > Default: `0`.
>
> > Import the `site` module at startup?
> >
> > If equal to zero, disable the import of the module site and the
> > site-dependent manipulations of `sys.path` that it entails.
> >
> > Also disable these manipulations if the `site` module is explicitly
> > imported later (call `site.main` if you want them to be triggered).
> >
> > Set to `0` by the `-S` command line option.
> >
> > `sys.flags.no_site` is set to the inverted value of
> > :c`~PyConfig.site_import`.
> >
> > Default: `1`.
>
> > If non-zero, skip the first line of the :c`PyConfig.run_filename`
> > source.
> >
> > It allows the usage of non-Unix forms of `#!cmd`. This is intended
> > for a DOS specific hack only.
> >
> > Set to `1` by the `-x` command line option.
> >
> > Default: `0`.
>
> > Encoding and encoding errors of `sys.stdin`, `sys.stdout` and
> > `sys.stderr` (but `sys.stderr` always uses `"backslashreplace"`
> > error handler).
> >
> > If :c`Py_SetStandardStreamEncoding` has been called, use its *error*
> > and *errors* arguments if they are not `NULL`.
> >
> > Use the `PYTHONIOENCODING` environment variable if it is non-empty.
> >
> > Default encoding:
> >
> > -   `"UTF-8"` if :c`PyPreConfig.utf8_mode` is non-zero.
> > -   Otherwise, use the `locale encoding`.
> >
> > Default error handler:
> >
> > -   On Windows: use `"surrogateescape"`.
> > -   `"surrogateescape"` if :c`PyPreConfig.utf8_mode` is non-zero, or
> >     if the LC_CTYPE locale is "C" or "POSIX".
> > -   `"strict"` otherwise.
>
> > Enable tracemalloc?
> >
> > If non-zero, call `tracemalloc.start` at startup.
> >
> > Set by `-X tracemalloc=N <-X>` command line option and by the
> > `PYTHONTRACEMALLOC` environment variable.
> >
> > Default: `-1` in Python mode, `0` in isolated mode.
>
> > Use `environment variables <using-on-envvars>`?
> >
> > If equals to zero, ignore the `environment variables
> > <using-on-envvars>`.
> >
> > Set to `0` by the `-E` environment variable.
> >
> > Default: `1` in Python config and `0` in isolated config.
>
> > If non-zero, add the user site directory to `sys.path`.
> >
> > Set to `0` by the `-s` and `-I` command line options.
> >
> > Set to `0` by the `PYTHONNOUSERSITE` environment variable.
> >
> > Default: `1` in Python mode, `0` in isolated mode.
>
> > Verbose mode. If greater than `0`, print a message each time a
> > module is imported, showing the place (filename or built-in module)
> > from which it is loaded.
> >
> > If greater than or equal to `2`, print a message for each file that
> > is checked for when searching for a module. Also provides
> > information on module cleanup at exit.
> >
> > Incremented by the `-v` command line option.
> >
> > Set by the `PYTHONVERBOSE` environment variable value.
> >
> > Default: `0`.
>
> > Options of the `warnings` module to build warnings filters, lowest
> > to highest priority: `sys.warnoptions`.
> >
> > The `warnings` module adds `sys.warnoptions` in the reverse order:
> > the last :c`PyConfig.warnoptions` item becomes the first item of
> > `warnings.filters` which is checked first (highest priority).
> >
> > The `-W` command line options adds its value to
> > :c`~PyConfig.warnoptions`, it can be used multiple times.
> >
> > The `PYTHONWARNINGS` environment variable can also be used to add
> > warning options. Multiple options can be specified, separated by
> > commas (`,`).
> >
> > Default: empty list.
>
> > If equal to `0`, Python won't try to write `.pyc` files on the
> > import of source modules.
> >
> > Set to `0` by the `-B` command line option and the
> > `PYTHONDONTWRITEBYTECODE` environment variable.
> >
> > `sys.dont_write_bytecode` is initialized to the inverted value of
> > :c`~PyConfig.write_bytecode`.
> >
> > Default: `1`.
>
> > Values of the `-X` command line options: `sys._xoptions`.
> >
> > Default: empty list.

If :c`~PyConfig.parse_argv` is non-zero, :c`~PyConfig.argv` arguments
are parsed the same way the regular Python parses `command line
arguments <using-on-cmdline>`, and Python arguments are stripped from
:c`~PyConfig.argv`.

The :c`~PyConfig.xoptions` options are parsed to set other options: see
the `-X` command line option.

3.9

The `show_alloc_count` field has been removed.

## Initialization with PyConfig

Function to initialize Python:

> Initialize Python from *config* configuration.

The caller is responsible to handle exceptions (error or exit) using
:c`PyStatus_Exception` and :c`Py_ExitStatusException`.

If :c`PyImport_FrozenModules`, :c`PyImport_AppendInittab` or
:c`PyImport_ExtendInittab` are used, they must be set or called after
Python preinitialization and before the Python initialization. If Python
is initialized multiple times, :c`PyImport_AppendInittab` or
:c`PyImport_ExtendInittab` must be called before each Python
initialization.

The current configuration (`PyConfig` type) is stored in
`PyInterpreterState.config`.

Example setting the program name:

``` c
void init_python(void)
{
    PyStatus status;

    PyConfig config;
    PyConfig_InitPythonConfig(&config);

    /* Set the program name. Implicitly preinitialize Python. */
    status = PyConfig_SetString(&config, &config.program_name,
                                L"/path/to/my_program");
    if (PyStatus_Exception(status)) {
        goto exception;
    }

    status = Py_InitializeFromConfig(&config);
    if (PyStatus_Exception(status)) {
        goto exception;
    }
    PyConfig_Clear(&config);
    return;

exception:
    PyConfig_Clear(&config);
    Py_ExitStatusException(status);
}
```

More complete example modifying the default configuration, read the
configuration, and then override some parameters. Note that since 3.11,
many parameters are not calculated until initialization, and so values
cannot be read from the configuration structure. Any values set before
initialize is called will be left unchanged by initialization:

``` c
PyStatus init_python(const char *program_name)
{
    PyStatus status;

    PyConfig config;
    PyConfig_InitPythonConfig(&config);

    /* Set the program name before reading the configuration
       (decode byte string from the locale encoding).

       Implicitly preinitialize Python. */
    status = PyConfig_SetBytesString(&config, &config.program_name,
                                     program_name);
    if (PyStatus_Exception(status)) {
        goto done;
    }

    /* Read all configuration at once */
    status = PyConfig_Read(&config);
    if (PyStatus_Exception(status)) {
        goto done;
    }

    /* Specify sys.path explicitly */
    /* If you want to modify the default set of paths, finish
       initialization first and then use PySys_GetObject("path") */
    config.module_search_paths_set = 1;
    status = PyWideStringList_Append(&config.module_search_paths,
                                     L"/path/to/stdlib");
    if (PyStatus_Exception(status)) {
        goto done;
    }
    status = PyWideStringList_Append(&config.module_search_paths,
                                     L"/path/to/more/modules");
    if (PyStatus_Exception(status)) {
        goto done;
    }

    /* Override executable computed by PyConfig_Read() */
    status = PyConfig_SetString(&config, &config.executable,
                                L"/path/to/my_executable");
    if (PyStatus_Exception(status)) {
        goto done;
    }

    status = Py_InitializeFromConfig(&config);

done:
    PyConfig_Clear(&config);
    return status;
}
```

## Isolated Configuration

:c`PyPreConfig_InitIsolatedConfig` and :c`PyConfig_InitIsolatedConfig`
functions create a configuration to isolate Python from the system. For
example, to embed Python into an application.

This configuration ignores global configuration variables, environment
variables, command line arguments (:c`PyConfig.argv` is not parsed) and
user site directory. The C standard streams (ex: `stdout`) and the
LC_CTYPE locale are left unchanged. Signal handlers are not installed.

Configuration files are still used with this configuration to determine
paths that are unspecified. Ensure :c`PyConfig.home` is specified to
avoid computing the default path configuration.

## Python Configuration

:c`PyPreConfig_InitPythonConfig` and :c`PyConfig_InitPythonConfig`
functions create a configuration to build a customized Python which
behaves as the regular Python.

Environments variables and command line arguments are used to configure
Python, whereas global configuration variables are ignored.

This function enables C locale coercion (`538`) and
`Python UTF-8 Mode <utf8-mode>` (`540`) depending on the LC_CTYPE
locale, `PYTHONUTF8` and `PYTHONCOERCECLOCALE` environment variables.

## Python Path Configuration

:c`PyConfig` contains multiple fields for the path configuration:

-   Path configuration inputs:
    -   :c`PyConfig.home`
    -   :c`PyConfig.platlibdir`
    -   :c`PyConfig.pathconfig_warnings`
    -   :c`PyConfig.program_name`
    -   :c`PyConfig.pythonpath_env`
    -   current working directory: to get absolute paths
    -   `PATH` environment variable to get the program full path (from
        :c`PyConfig.program_name`)
    -   `__PYVENV_LAUNCHER__` environment variable
    -   (Windows only) Application paths in the registry under
        "SoftwarePythonPythonCoreX.YPythonPath" of HKEY_CURRENT_USER and
        HKEY_LOCAL_MACHINE (where X.Y is the Python version).
-   Path configuration output fields:
    -   :c`PyConfig.base_exec_prefix`
    -   :c`PyConfig.base_executable`
    -   :c`PyConfig.base_prefix`
    -   :c`PyConfig.exec_prefix`
    -   :c`PyConfig.executable`
    -   :c`PyConfig.module_search_paths_set`,
        :c`PyConfig.module_search_paths`
    -   :c`PyConfig.prefix`

If at least one "output field" is not set, Python calculates the path
configuration to fill unset fields. If
:c`~PyConfig.module_search_paths_set` is equal to `0`,
:c`~PyConfig.module_search_paths` is overridden and
:c`~PyConfig.module_search_paths_set` is set to `1`.

It is possible to completely ignore the function calculating the default
path configuration by setting explicitly all path configuration output
fields listed above. A string is considered as set even if it is
non-empty. `module_search_paths` is considered as set if
`module_search_paths_set` is set to `1`. In this case,
`module_search_paths` will be used without modification.

Set :c`~PyConfig.pathconfig_warnings` to `0` to suppress warnings when
calculating the path configuration (Unix only, Windows does not log any
warning).

If :c`~PyConfig.base_prefix` or :c`~PyConfig.base_exec_prefix` fields
are not set, they inherit their value from :c`~PyConfig.prefix` and
:c`~PyConfig.exec_prefix` respectively.

:c`Py_RunMain` and :c`Py_Main` modify `sys.path`:

-   If :c`~PyConfig.run_filename` is set and is a directory which
    contains a `__main__.py` script, prepend :c`~PyConfig.run_filename`
    to `sys.path`.
-   If :c`~PyConfig.isolated` is zero:
    -   If :c`~PyConfig.run_module` is set, prepend the current
        directory to `sys.path`. Do nothing if the current directory
        cannot be read.
    -   If :c`~PyConfig.run_filename` is set, prepend the directory of
        the filename to `sys.path`.
    -   Otherwise, prepend an empty string to `sys.path`.

If :c`~PyConfig.site_import` is non-zero, `sys.path` can be modified by
the `site` module. If :c`~PyConfig.user_site_directory` is non-zero and
the user's site-package directory exists, the `site` module appends the
user's site-package directory to `sys.path`.

The following configuration files are used by the path configuration:

-   `pyvenv.cfg`
-   `._pth` file (ex: `python._pth`)
-   `pybuilddir.txt` (Unix only)

If a `._pth` file is present:

-   Set :c`~PyConfig.isolated` to `1`.
-   Set :c`~PyConfig.use_environment` to `0`.
-   Set :c`~PyConfig.site_import` to `0`.
-   Set :c`~PyConfig.safe_path` to `1`.

The `__PYVENV_LAUNCHER__` environment variable is used to set
:c`PyConfig.base_executable`

## Py_RunMain()

> Execute the command (:c`PyConfig.run_command`), the script
> (:c`PyConfig.run_filename`) or the module (:c`PyConfig.run_module`)
> specified on the command line or in the configuration.
>
> By default and when if `-i` option is used, run the REPL.
>
> Finally, finalizes Python and returns an exit status that can be
> passed to the `exit()` function.

See `Python Configuration <init-python-config>` for an example of
customized Python always running in isolated mode using :c`Py_RunMain`.

## Py_GetArgcArgv()

> Get the original command line arguments, before Python modified them.
>
> See also :c`PyConfig.orig_argv` member.

## Multi-Phase Initialization Private Provisional API

This section is a private provisional API introducing multi-phase
initialization, the core feature of `432`:

-   "Core" initialization phase, "bare minimum Python":
    -   Builtin types;
    -   Builtin exceptions;
    -   Builtin and frozen modules;
    -   The `sys` module is only partially initialized (ex: `sys.path`
        doesn't exist yet).
-   "Main" initialization phase, Python is fully initialized:
    -   Install and configure `importlib`;
    -   Apply the `Path Configuration <init-path-config>`;
    -   Install signal handlers;
    -   Finish `sys` module initialization (ex: create `sys.stdout` and
        `sys.path`);
    -   Enable optional features like `faulthandler` and `tracemalloc`;
    -   Import the `site` module;
    -   etc.

Private provisional API:

-   :c`PyConfig._init_main`: if set to `0`, :c`Py_InitializeFromConfig`
    stops at the "Core" initialization phase.
-   :c`PyConfig._isolated_interpreter`: if non-zero, disallow threads,
    subprocesses and fork.

> Move to the "Main" initialization phase, finish the Python
> initialization.

No module is imported during the "Core" phase and the `importlib` module
is not configured: the `Path Configuration <init-path-config>` is only
applied during the "Main" phase. It may allow to customize Python in
Python to override or tune the `Path Configuration <init-path-config>`,
maybe install a custom `sys.meta_path` importer or an import hook, etc.

It may become possible to calculatin the `Path Configuration
<init-path-config>` in Python, after the Core phase and before the Main
phase, which is one of the `432` motivation.

The "Core" phase is not properly defined: what should be and what should
not be available at this phase is not specified yet. The API is marked
as private and provisional: the API can be modified or even be removed
anytime until a proper public API is designed.

Example running Python code between "Core" and "Main" initialization
phases:

``` c
void init_python(void)
{
    PyStatus status;

    PyConfig config;
    PyConfig_InitPythonConfig(&config);
    config._init_main = 0;

    /* ... customize 'config' configuration ... */

    status = Py_InitializeFromConfig(&config);
    PyConfig_Clear(&config);
    if (PyStatus_Exception(status)) {
        Py_ExitStatusException(status);
    }

    /* Use sys.stderr because sys.stdout is only created
       by _Py_InitializeMain() */
    int res = PyRun_SimpleString(
        "import sys; "
        "print('Run Python code before _Py_InitializeMain', "
               "file=sys.stderr)");
    if (res < 0) {
        exit(1);
    }

    /* ... put more configuration code here ... */

    status = _Py_InitializeMain();
    if (PyStatus_Exception(status)) {
        Py_ExitStatusException(status);
    }
}
```